<img src="https://api.immobilienscout24.de/content/dam/is24/images/api/api-logo-immobilienscout24_250x50_font_blau.png" alt="In Zusammenarbeit mit Immobilienscout24.de">

# Crawl real estate data from Immobilienscout24.de

Notes
* The SEARCH API will be used
* API permission was granted because data will be used for a student project


*API Documentation: https://api.immobilienscout24.de/*

## Setup

In [ ]:
import pandas as pd
import numpy as np
from xml.etree import ElementTree
import getpass

import requests
from requests_oauthlib import OAuth1

# define path where output file should be exported to
#CB: 
#path_op = '/Users/cbineyti/Library/Mobile Documents/com~apple~CloudDocs/Docs/40_DataScience/80_Ironhack/30_Testing/IS24/'

# auth information obtained from IS24
client_key = 'immokb24_contentKey'
client_secret = getpass.getpass() 

base_url = 'https://rest.immobilienscout24.de/restapi/api/search/v1.0/search/region'

auth = OAuth1(client_key, client_secret)  # , 'USER_OAUTH_TOKEN', 'USER_OAUTH_TOKEN_SECRET')


Notebook display settings:

In [ ]:
pd.options.display.max_columns = None
pd.options.display.max_rows = 7000
pd.options.display.max_colwidth = 200

## Set parameters for different requests 

In [ ]:
# real estate objects to buy IN GERMANY
params_de = { 
    'realestatetype':'apartmentbuy', 
    'geocodes': 1276
}

# real estate objects to buy IN ESSEN
params_essen = {
    'realestatetype':'apartmentbuy', 
    'geocodes': 1276010015
}

# ... Berlin
params_bln = {
    'realestatetype':'apartmentbuy', 
    'geocodes': 1276003001
}

params_ham = {
    'realestatetype':'apartmentbuy', 
    'geocodes': 1276006001 
}


## Execute requests

In [ ]:
# Germany
resp_de = requests.get(base_url, auth=auth, params=params_de)

In [ ]:
# Essen
resp_essen = requests.get(base_url, auth=auth, params=params_essen)

In [ ]:
# Berlin
resp_bln = requests.get(base_url, auth=auth, params=params_bln)

In [ ]:
# Hamburg
resp_ham = requests.get(base_url, auth=auth, params=params_ham)

In [ ]:
#print(resp_de.text)

In [ ]:
#print(resp_essen.text)

In [ ]:
#print(resp_bln.text)

# Read XML into Dataframe

#### Read XML output into Element Tree root

In [ ]:
root_de = ElementTree.fromstring(resp_de.content)

In [ ]:
root_essen = ElementTree.fromstring(resp_essen.content)

# Extract information from one page

#### Define function to get information from a root into a dictionary
* Input: root
* Output: dictionary

### <font color="red">USE YIELD / recursiv function</font>

## <font color="Green"><b> Export to Library.</b> </font>

In [ ]:
# Define function to get information from a root into a dictionary
def get_xml_data(root):
    
    # define lists for each field that will be extracted
    lst_realEstateId = []
    lst_titles = []
    lst_cities = []
    lst_quarters = []
    lst_prices = []
    lst_livingSpace = []
    lst_rooms = []
    lst_num_pages = []

    # loop through the real estate entries and save information into lists
    for child in root[1]:
        lst_realEstateId.append(child.find('realEstateId').text)
        lst_titles.append(child[3].find('title').text)
        lst_cities.append(child[3][1].find('city').text)
        lst_quarters.append(child[3][1].find('quarter').text)
        lst_prices.append(child[3].find('price')[0].text)
        lst_livingSpace.append(child[3].find('livingSpace').text)
        lst_rooms.append(child[3].find('numberOfRooms').text) 
        lst_num_pages.append(root[0].find('pageNumber').text)

    # create dictionary from lists above
    dict_lsts = {'id': lst_realEstateId,
                 'title': lst_titles,
                 'city': lst_cities,
                 'quarter': lst_quarters,
                 'price': lst_prices,
                 'qm': lst_livingSpace,
                 'n_room': lst_rooms,
                 'n_page': lst_num_pages
                }
    
    return dict_lsts

#### Test the function `get_xml_data`

In [ ]:
df_essen = pd.DataFrame(get_xml_data(root_essen)).assign(is24_page=1)


In [ ]:
df_essen;

#### Test the function in a loop

In [ ]:
for i in range(3):
    df_essen = df_essen.append(pd.DataFrame(get_xml_data(root_essen)).assign(is24_page=i+2))

In [ ]:
df_essen.reset_index(drop=True);

# Loop over all pages of the response (XML) 

Steps:

* define a function that returns a dataframe based on a URL
* define a function that returns the number of pages based on a URL
* loop over all pages of an XML response and concatinate all output into one dataframe

#### Function to get a dataframe based on a URL

## <font color="Green"><b> Export to Library.</b> </font>

In [ ]:
def df_from_url(url, params):  
    # send the get request using global auth information and provided url and params
    global auth
    resp = requests.get(url, auth=auth, params=params)
    
    # get XML response into the root of an ElementTree
    root = ElementTree.fromstring(resp.content)
    
    # call get_xml_data function to store information into a dataframe
    df = pd.DataFrame(get_xml_data(root))
    
    return df


#### Function to get the number of pages based on a URL

## <font color="Green"><b> Export to Library.</b> </font>

In [ ]:
def num_pages_from_url(url, params):  
    
    # send the get request using global auth information and provided url and params
    global auth
    resp = requests.get(url, auth=auth, params=params)
    
    # get XML response into the root of an ElementTree
    root = ElementTree.fromstring(resp.content)
    
    # navigate to the number of pages and get the text
    num_pages = int(root[0].find('numberOfPages').text)
    
    return num_pages


#### Test `num_pages_from_url` function

In [ ]:
num_pages_de = num_pages_from_url(base_url, params_de)
num_pages_de

#### Test `df_from_url` function

In [ ]:
df_essen = df_from_url(base_url, params_essen)


In [ ]:
params_essen

In [ ]:
df_essen;

### Create `df_essen`

#### Steps:

* create `df_essen` with first page of response
* loop over all other pages and append dataframe built from each page to the existing dataframe

In [ ]:
df_essen = df_from_url(base_url, params_essen)

for i in range(2, num_pages_from_url(base_url, params_essen) + 1):
    params_essen_loop = { 
        'realestatetype':'apartmentbuy', 
        'geocodes': 1276010015,
        'pagenumber': i
    }
    df_essen = df_essen.append(df_from_url(base_url, params_essen_loop))

### Write function to create concatenated dataframe

#### Define dictionary with location name (city or country or region, ...) and corresponding geocodes

The geocodes can be obtained using the GIS API from api.immobilienscout24.de

## <font color="Green"><b> Export to Library.</b> </font>

In [ ]:
di_locations = {
    'deutschland': 1276,
    'berlin': 1276003001,
    'essen': 1276010015,
    'hamburg': 1276006001
}

test dictionary...

In [ ]:
location = 'essen'

In [ ]:
di_locations[location];

In [ ]:
di_locations;

#### Define function to create entire dataframe for a location

## <font color="Green"><b> Export to Library.</b> </font>

In [ ]:
def create_df_all(url, location, di_locations):
    
    params_loc = { 
        'realestatetype':'apartmentbuy', 
        'geocodes': di_locations[location]
    }
    
    df_location = df_from_url(url, params_loc)
    
    for i in range(2, num_pages_from_url(url, params_loc) + 1):
        params_loc_loop = { 
            'realestatetype':'apartmentbuy', 
            'geocodes': di_locations[location],
            'pagenumber': i
        }
        df_location = df_location.append(df_from_url(url, params_loc_loop))
    
    return df_location

### Calculate additional columns

#### Calculate price per qm in new column `price_per_qm`

In [ ]:
df_essen['price'] = pd.to_numeric(df_essen['price'])
df_essen['qm'] = pd.to_numeric(df_essen['qm'])
df_essen['price_per_qm'] = round(df_essen['price'] / df_essen['qm'], 1)

#### Add column `zwangsversteigerung` to indicate whether the listed item is a Zwangsversteigerung

*Note: this piece of information is not reliable*

In [ ]:
df_essen['zwangsversteigerung'] = np.where(df_essen['title'].str.contains('Zwangs'), 'yes', 'no')

#### Reset index and sort dataframe by price per qm

In [ ]:
df_essen_op = df_essen.reset_index(drop=True).sort_values(by='price_per_qm')

# Output `df_essen` to CSV file

In [ ]:
df_essen_op;

In [ ]:
# df_essen_op.to_csv(path_op + 'essen_v0.1.csv', index=False)

# EDA

In [ ]:
df = df_essen_op.copy()

In [ ]:
df.price_per_qm.hist();

---

# Create Berlin dataframe `df_berlin`

Steps:

* Run `create_df_all` to create dataframe containing all information (i.e. all pages) for Berlin
* Calculate the `price_per_qm`
* Save output to csv file

In [ ]:
df_berlin = create_df_all(base_url, 'berlin', di_locations).reset_index(drop=True)

### Data Wrangling

In [ ]:
df_berlin.head(4000)

In [ ]:
df_berlin[["id"]].astype(int);

<font color="red"><b>TODO:</b> add a function that takes in a dataframe and a list of column names and returns a dataframe where the provided column names are transformed into numeric data type.</font>

Currently, this is manually done in the `calc_price_per_qm` function below

In [ ]:
# input df has to have the columns 'price' and 'qm'
def calc_price_per_qm(df):

    df['price'] = pd.to_numeric(df['price'])
    df['qm'] = pd.to_numeric(df['qm'])
    df['n_room'] = pd.to_numeric(df['n_room'])
    df['n_page'] = pd.to_numeric(df['n_page'])
    df['price_per_qm'] = np.where(df['qm'] != 0, round(df['price'] / df['qm'], 1), np.NaN)
    
    return df

In [ ]:
df_berlin = calc_price_per_qm(df_berlin)

In [ ]:
df_berlin.head(3)

In [ ]:
df_berlin.info()

### Save `df_berlin` to csv file

In [ ]:
df_berlin.to_csv('20200122_apts_buy_berlin_v0.2_AG_test.csv')

In [ ]:
df_berlin_ag_test = pd.read_csv('20200122_apts_buy_berlin_v0.2_AG_test.csv')

In [ ]:
df_berlin_ag_test[["id"]].astype(int);

---

# EDA with `df_berlin`

#### Check whether numeric columns have numeric dtypes

In [ ]:
df_berlin.iloc[:, 4:].info()

#### Check the distribution of `price_per_qm`

In [ ]:
bins_p_per_qm = []
for i in range(30):
    bins_p_per_qm.append(i*500)

bins_p_per_qm

In [ ]:
df_berlin['price_per_qm'].hist(bins=bins_p_per_qm);

In [ ]:
bins_p = []
for i in range(100):
    bins_p.append(i*10000)

In [ ]:
df_berlin['price'].hist(bins=bins_p);